# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

In [16]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


## Question 1: What is the sum of the outputs of the generator for limit = 5?

In [20]:
def sum_of_squares_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 5
sum_generator = sum_of_squares_generator(limit)

sum_outputs = sum(sum_generator)
print(f'The sum of the limit 5 is: {sum_outputs}')

The sum of the limit 5 is: 8.382332347441762


## Question 2: What is the 13th number yielded

In [3]:
def square_root_generator(limit):
    n = 13
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(f'The thirteenth square root: {sqrt_value}')

The thirteenth square root: 3.605551275463989


# 2. Append a generator to a table with existing data
Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

In [7]:
# Generator 1
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

# generator 2
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [4]:
import dlt
import duckdb

### 1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.


In [8]:
# Create DLT Generators Pipeline
generators_pipeline = dlt.pipeline(destination='duckdb',
                                   dataset_name='persons')

In [9]:
# Load people_1 generator
info = generators_pipeline.run(people_1(),
                               table_name="people",
                               write_disposition="replace")
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.48 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons
The duckdb destination used duckdb:////home/claytor/my_repo/03a - workshop - data-ingestion/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707789632.3365998 is LOADED and contains no failed jobs


In [28]:
# connecting to duckdb
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the first batch of people
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print("DLT LOADS:")
display(conn.sql("show tables"))

# full query of people table
half_people = conn.sql("SELECT * FROM people").df()
print("🦆 Duck Table 🦆")
display(half_people)

# full query of people table
peoples_ages = conn.sql("SELECT SUM(age) as sum_of_ages FROM people").df()
print("🦆 The sum of ages from people_1 generator 🦆")
display(peoples_ages)



DLT LOADS:


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

🦆 Duck Table 🦆


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1707789632.3365998,wAWJsfW9ipbZkQ
1,2,Person_2,27,City_A,1707789632.3365998,vCKqzG+cwuQQlw
2,3,Person_3,28,City_A,1707789632.3365998,PzJzH2Dm+lzeCQ
3,4,Person_4,29,City_A,1707789632.3365998,7txsnXeahi2ChA
4,5,Person_5,30,City_A,1707789632.3365998,8km76TV08akttg


🦆 The sum of ages from people_1 generator 🦆


,sum_of_ages
0,140.0


### 2. Append the second generator to the same table as the first.


In [29]:
# Append people_2 generator
info = generators_pipeline.run(people_2(),
                               table_name="people",
                               write_disposition="append")
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.25 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons
The duckdb destination used duckdb:////home/claytor/my_repo/03a - workshop - data-ingestion/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707791516.4016333 is LOADED and contains no failed jobs


In [33]:
# Appended People
full_people = conn.sql("SELECT * FROM people").df()
print("🦆🦆 Appended Ducks! 🦆🦆")
display(full_people)

🦆🦆 Appended Ducks! 🦆🦆


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707789632.3365998,wAWJsfW9ipbZkQ,None
1,2,Person_2,27,City_A,1707789632.3365998,vCKqzG+cwuQQlw,None
2,3,Person_3,28,City_A,1707789632.3365998,PzJzH2Dm+lzeCQ,None
3,4,Person_4,29,City_A,1707789632.3365998,7txsnXeahi2ChA,None
4,5,Person_5,30,City_A,1707789632.3365998,8km76TV08akttg,None
5,3,Person_3,33,City_B,1707791516.4016333,cLNqqZyg+4iUzg,Job_3
6,4,Person_4,34,City_B,1707791516.4016333,q984KxN2uRVniw,Job_4
7,5,Person_5,35,City_B,1707791516.4016333,tlqT+QPNHnnKRg,Job_5
8,6,Person_6,36,City_B,1707791516.4016333,fGZyTb0nqmWuYg,Job_6
9,7,Person_7,37,City_B,1707791516.4016333,yjEDayouQkgK9A,Job_7


### 3. **After correctly appending the data, calculate the sum of all ages of people.**




In [34]:
# full query of people table
appended_ages = conn.sql("SELECT SUM(age) as appendages FROM people").df()
print("🫲🦆🫱 Appendages, get it? 🫲🦆🫱")
display(appended_ages)

🫲🦆🫱 Appendages, get it? 🫲🦆🫱


,appendages
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2. A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.
1) Load your first generator
2) Load the second one with merge. 

Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.
> After loading, you should have a total of 8 records, and ID 3 should have age 33.

#### Calculate the sum of ages of all the people loaded as described above.

In [36]:
# Keyed up people
info = generators_pipeline.run(people_1(),
                               table_name="merged_people",
                               write_disposition="replace",
                               primary_key="id")

In [39]:
# Merged people
info = generators_pipeline.run(people_2(),
                               table_name="merged_people",
                               write_disposition="merge",
                               primary_key='id')
print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset persons
The duckdb destination used duckdb:////home/claytor/my_repo/03a - workshop - data-ingestion/dlt_resources/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707793735.5774221 is LOADED and contains no failed jobs


In [41]:
merged_people = conn.sql("SELECT * FROM merged_people").df()
print("🤼 Merged People! 🤼")
display(merged_people)

merged_ages = conn.sql("SELECT SUM(age) as merged_ages FROM merged_people").df()
print("👴👶 Merged Ages! 👶👴")
display(merged_ages)

🤼 Merged People! 🤼


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707793276.2888238,cnOac0GUMv9USg,None
1,2,Person_2,27,City_A,1707793276.2888238,CDFsc8Wwrdnz2A,None
2,8,Person_8,38,City_B,1707793735.5774221,XXWvtWmkd/tvFg,Job_8
3,4,Person_4,34,City_B,1707793735.5774221,yGWSqYtaXvgUWA,Job_4
4,5,Person_5,35,City_B,1707793735.5774221,+yCqA4x3KzrKOA,Job_5
5,3,Person_3,33,City_B,1707793735.5774221,rFkrtN+YgAi+Bg,Job_3
6,6,Person_6,36,City_B,1707793735.5774221,2hBrj05BWaI3DA,Job_6
7,7,Person_7,37,City_B,1707793735.5774221,tnCUWqL025nlrw,Job_7


👴👶 Merged Ages! 👶👴


,merged_ages
0,266.0
